In [5]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = "1xnMYOD5FkrwVKakAAaAaqOyD"
        consumer_secret = "hf1ADLyEPWll1rl43sGHdwl7eMWmK1LbzU6ajSxQKbwB86I15D"
        access_token = "732181352326434816-XpBDVRPACxdBtChDFqxVhH2bz5nRGm0"
        access_token_secret = "2tu6wcnVBTy6aZNtNjYrSmzzJv1pAvGI2YbTNgjXPc8rv"
        

        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return (' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()))

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []

        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)

            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            # return parsed tweets
            return tweets

        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))




In [ ]:
import pandas as pd

def tweetListMaker(tweetsList,sentimentNumber):
    processedTweetsList = []
    for i in tweetsList:
        #print("Positive")

        #print(type(i),i)
        if isinstance(i,dict):
            processedTweetsList.append([i['text'],sentimentNumber])
        elif isinstance(i,list):
            for j in i:
                print(j)
                processedTweetsList.append([j['text'],sentimentNumber])
    
    return (processedTweetsList)
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    
    myFilters = ["cash flow", "balance sheet", "stock", "prices", "deprecation"]
    ptweets = []
    ntweets = []
    neuTweets = []
    
    myDf = pd.DataFrame()
    for currentFilter in myFilters:
        # calling function to get tweets
        tweets = api.get_tweets(query = currentFilter, count = 2000)

        # picking positive tweets from tweets
        print(len(ptweets),len(ntweets),len(neuTweets))
        if len(ptweets) == 0:
            ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
        else:
            ptweets.append([tweet for tweet in tweets if tweet['sentiment'] == 'positive'])
        # percentage of positive tweets
        print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
        # picking negative tweets from tweets
        
        if len(ntweets) == 0:
            ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
        else:
            ntweets.append([tweet for tweet in tweets if tweet['sentiment'] == 'negative'])
        # percentage of negative tweets
        print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))

        neut = [tweets not in ptweets]
        
        if len(neuTweets) == 0:
            neuTweets = [neut not in ntweets]
        else:
            neuTweets.append([neut not in ntweets])
        
        print(len(neuTweets))
        # percentage of neutral tweets
        #print("Neutral tweets percentage: {} %".format(100*len(tweets - ntweets - ptweets)/len(tweets)))

        # printing first 5 positive tweets
        #print("\n\nPositive tweets:")
        #for tweet in ptweets[:10]:
        #    print(tweet['text'])

        # printing first 5 negative tweets
        #print("\n\nNegative tweets:")
        #for tweet in ntweets[:10]:
        #    print(tweet['text'])
    
    a = [t for t in ptweets]
    #print(a)
    #myDf = pd.DataFrame([[t['text'],3] for t in ptweets],columns=['tweet','label'])
    #myDf = myDf.append(pd.DataFrame([[t['text'],3] for t in ntweets],columns=['tweet','label']))
    #myDf = myDf.append(pd.DataFrame([[t['text'],3] for t in neuTweets],columns=['tweet','label']))
    pt = tweetListMaker(ptweets,3)
    nt = tweetListMaker(ptweets,1)
    neut = tweetListMaker(neuTweets,2)
    #myDf['label'].iloc[currentLen:]  = 2
    
    myDf = pd.DataFrame(pt,columns=['tweet','label'])
    myDf = myDf.append(pd.DataFrame(nt,columns=['tweet','label']))
    myDf = myDf.append(pd.DataFrame(neut,columns=['tweet','label']))
    currentLen = len(myDf)
    
    
    return (myDf)


#print(ptweets)
#myDf['tweets'] = ptweets
#myDf['label']  = 3


myDf = main()
positiveTweets = []
for i in ptweets:
    print("Positive")
    
    #print(type(i),i)
    if isinstance(i,dict):
        positiveTweets.append([i['text'],3])
    elif isinstance(i,list):
        for j in i:
            positiveTweets.append([j['text'],3])
positiveTweets

(0, 0, 0)
Positive tweets percentage: 40 %
Negative tweets percentage: 12 %
1


In [2]:
!pip install tweepy
!pip install textblob

  Using cached textblob-0.13.1-py2.py3-none-any.whl


In [3]:
myDf['tweets'].iloc[0]

NameError: name 'myDf' is not defined

In [56]:
len(myDf)

32

In [61]:
pd.DataFrame({"sahil",2},columns=['tweets','label'])

,tweets,label
0,sahil,2
